In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [57]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from functions import*

c:\Users\zhouq\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
import os
path=os.path.dirname(os.getcwd())

In [52]:
K=200
e=0.01

var_list=['hoursperweek','age','capitalgain','capitalloss' ,'education-num'] #
var_dim=len(var_list)
pa='race' #'sex'
pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
pd.set_option('future.no_silent_downcasting', True)

messydata=pd.read_csv(path+'/data/adult_csv.csv',usecols=var_list+[pa,'class'])
messydata=messydata.rename(columns={pa:'S','class':'Y'})
messydata['S']=messydata['S'].replace(pa_dict)
messydata['Y']=messydata['Y'].replace({'>50K':1,'<=50K':0})
messydata=messydata[(messydata['S']==1)|(messydata['S']==0)]
for col in var_list+['S','Y']:
    messydata[col]=messydata[col].astype('category')
messydata['W']=1
X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]

In [53]:
tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
    dist=rdata_analysis(messydata,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
x_list=[]
for key,val in tv_dist.items():
    if val>0.08:
        x_list+=[key]

In [55]:
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    clf=RandomForestClassifier(max_depth=5, random_state=0).fit(X_train[:,0:var_dim],y_train)

    df_test=pd.DataFrame(np.concatenate((X_test,y_test.reshape(-1,1)), axis=1),columns=var_list+['S','W','Y'])
    df_test=df_test.groupby(by=var_list+['S','Y'],as_index=False).sum()

    if len(x_list)>1:
        df_test['X']=[tuple(df_test[x_list].values[r]) for r in range(df_test.shape[0])]
        x_range=sorted(set(df_test['X']))
        weight=list(1/(df_test[x_list].max()-df_test[x_list].min())) # because 'education-num' range from 1 to 16 while others 1 to 4
        C=c_generate_higher(x_range,weight)
    else:
        df_test['X']=df_test[x_list]
        x_range=sorted(set(df_test['X']))
        C=c_generate(x_range)

    bin=len(x_range)
    var_range=list(pd.pivot_table(df_test,index=var_list,values=['S','W','Y']).index)
    dist=rdata_analysis(df_test,x_range,'X')
    dist['t_x']=dist['x'] # #dist['x'] #dist['x_0']*0.5+dist['x_1']*0.5 
    dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]
    px=np.matrix(dist['x']).T
    ptx=np.matrix(dist['t_x']).T
    if np.any(dist['x_0']==0): 
        p0=np.matrix((dist['x_0']+1.0e-9)/sum(dist['x_0']+1.0e-9)).T
    else:
        p0=np.matrix(dist['x_0']).T 
    if np.any(dist['x_1']==0):
        p1=np.matrix((dist['x_1']+1.0e-9)/sum(dist['x_1']+1.0e-9)).T
    else:
        p1=np.matrix(dist['x_1']).T 
    V=np.matrix(dist['v']).T

    coupling_base=baseline(C,e,px,ptx,V,K)
    coupling_bary=baseline(C,e,p0,p1,V,K)
    coupling_part2=partial_repair(C,e,px,ptx,V,1.0e-3,K)
    # coupling_part3=partial_repair(C,e,px,ptx,V,1.0e-3,K)
    # coupling_total=partial_repair(C,e,px,ptx,V,1.0e-5,K)
    # test_RW=reweighting(df_test)
    
    tv_base=assess_tv(df_test,coupling_base,x_range,x_list,var_list)
    tv_part2=assess_tv(df_test,coupling_part2,x_range,x_list,var_list)
    # tv_part3=assess_tv(df_test,coupling_part3,x_range,x_list,var_list)
    # tv_total=assess_tv(df_test,coupling_total,x_range,x_list,var_list)
    # tv_RW=assess_tv(test_RW,[],x_range,x_list,var_list)

    y_pred=clf.predict(np.array(df_test[var_list]))
    y_pred_base=postprocess(df_test,coupling_base,x_list,x_range,var_list,var_range,clf)
    y_pred_bary,tv_bary=postprocess_bary(df_test,coupling_bary,x_list,x_range,var_list,var_range,clf)
    y_pred_part2=postprocess(df_test,coupling_part2,x_list,x_range,var_list,var_range,clf)
    # y_pred_part3=postprocess(df_test,coupling_part3,x_list,x_range,var_list,var_range,clf)
    # y_pred_total=postprocess(df_test,coupling_total,x_list,x_range,var_list,var_range,clf)
    # y_pred_RW=clf.predict(np.array(test_RW[var_list]))

    new_row=pd.Series({'DI':DisparateImpact_postprocess(df_test,y_pred),
                        'f1 macro':f1_score(df_test['Y'], y_pred, average='macro',sample_weight=df_test['W']),
                        'f1 micro':f1_score(df_test['Y'], y_pred, average='micro',sample_weight=df_test['W']),
                        'f1 weighted':f1_score(df_test['Y'], y_pred, average='weighted',sample_weight=df_test['W']),
                        'TV distance':sum(abs(dist['x_0']-dist['x_1']))/2,'method':'origin'})
    new_row_base=pd.Series({'DI':DisparateImpact_postprocess(df_test,y_pred_base),
                        'f1 macro':f1_score(df_test['Y'], y_pred_base, average='macro',sample_weight=df_test['W']),
                        'f1 micro':f1_score(df_test['Y'], y_pred_base, average='micro',sample_weight=df_test['W']),
                        'f1 weighted':f1_score(df_test['Y'], y_pred_base, average='weighted',sample_weight=df_test['W']),
                        'TV distance':tv_base,'method':'baseline'})
    new_row_bary=pd.Series({'DI':DisparateImpact_postprocess(df_test,y_pred_bary),
                        'f1 macro':f1_score(df_test['Y'], y_pred_bary, average='macro',sample_weight=df_test['W']),
                        'f1 micro':f1_score(df_test['Y'], y_pred_bary, average='micro',sample_weight=df_test['W']),
                        'f1 weighted':f1_score(df_test['Y'], y_pred_bary, average='weighted',sample_weight=df_test['W']),
                        'TV distance':tv_bary,'method':'barycentre'})
    new_row_part2=pd.Series({'DI':DisparateImpact_postprocess(df_test,y_pred_part2),
                        'f1 macro':f1_score(df_test['Y'], y_pred_part2, average='macro',sample_weight=df_test['W']),
                        'f1 micro':f1_score(df_test['Y'], y_pred_part2, average='micro',sample_weight=df_test['W']),
                        'f1 weighted':f1_score(df_test['Y'], y_pred_part2, average='weighted',sample_weight=df_test['W']),
                        'TV distance':tv_part2,'method':'partial repair2'})
    # new_row_part3=pd.Series({'DI':DisparateImpact_postprocess(df_test,y_pred_part3),
    #                     'f1 macro':f1_score(df_test['Y'], y_pred_part3, average='macro',sample_weight=df_test['W']),
    #                     'f1 micro':f1_score(df_test['Y'], y_pred_part3, average='micro',sample_weight=df_test['W']),
    #                     'f1 weighted':f1_score(df_test['Y'], y_pred_part3, average='weighted',sample_weight=df_test['W']),
    #                     'TV distance':tv_part3,'method':'partial repair3'})
    # new_row_total=pd.Series({'DI':DisparateImpact_postprocess(df_test,y_pred_total),
    #                     'f1 macro':f1_score(df_test['Y'], y_pred_total, average='macro',sample_weight=df_test['W']),
    #                     'f1 micro':f1_score(df_test['Y'], y_pred_total, average='micro',sample_weight=df_test['W']),
    #                     'f1 weighted':f1_score(df_test['Y'], y_pred_total, average='weighted',sample_weight=df_test['W']),
    #                     'TV distance':tv_total,'method':'total repair'})
    # new_row_RW=pd.Series({'DI':DisparateImpact_postprocess(test_RW,y_pred_RW),
    #                     'f1 macro':f1_score(test_RW['Y'], y_pred_RW, average='macro',sample_weight=test_RW['W']),
    #                     'f1 micro':f1_score(test_RW['Y'], y_pred_RW, average='micro',sample_weight=test_RW['W']),
    #                     'f1 weighted':f1_score(test_RW['Y'], y_pred_RW, average='weighted',sample_weight=test_RW['W']),
    #                     'TV distance':tv_RW,'method':'reweighting'})
    
    report = pd.concat([report,new_row.to_frame().T,new_row_base.to_frame().T,new_row_bary.to_frame().T,new_row_part2.to_frame().T], ignore_index=True)
    #report = pd.concat([report,new_row.to_frame().T,new_row_base.to_frame().T,new_row_part2.to_frame().T,new_row_part3.to_frame().T,new_row_part4.to_frame().T], ignore_index=True) #,new_row_part4.to_frame().T
    # report = pd.concat([report,new_row.to_frame().T,new_row_base.to_frame().T,new_row_bary.to_frame().T,new_row_part3.to_frame().T,new_row_total.to_frame().T], ignore_index=True) #new_row_part2.to_frame().T,
    #report = pd.concat([report,new_row.to_frame().T,new_row_base.to_frame().T,new_row_bary.to_frame().T,new_row_part2.to_frame().T,new_row_part3.to_frame().T,new_row_part4.to_frame().T], ignore_index=True) #,new_row_part4.to_frame().T

In [56]:
report

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.353593,0.677809,0.81619,0.786812,0.19143,origin
1,0.353593,0.677809,0.81619,0.786812,0.191351,baseline
2,0.73633,0.535094,0.702298,0.679023,0.000049,barycentre
3,0.982744,0.627093,0.707142,0.716284,0.025413,partial repair2
4,0.679819,0.675185,0.814967,0.785183,0.150419,reweighting
5,0.44555,0.690283,0.823241,0.797115,0.195836,origin
6,0.44555,0.690283,0.823241,0.797115,0.195673,baseline
7,0.931475,0.54916,0.701222,0.687002,0.00001,barycentre
8,0.872677,0.6561,0.753431,0.752417,0.025543,partial repair2
9,0.77549,0.688621,0.822451,0.79609,0.160632,reweighting


In [51]:
report

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.449827,0.678789,0.817833,0.789005,0.205545,origin
1,0.449827,0.678789,0.817833,0.789005,0.205545,baseline
2,1.04678,0.547657,0.641296,0.654991,0.000075,barycentre
3,0.75701,0.658003,0.786661,0.7674,0.012038,partial repair2
4,0.895749,0.677025,0.816853,0.787855,0.16003,reweighting
5,0.405739,0.678773,0.817219,0.787287,0.210459,origin
6,0.405739,0.678773,0.817219,0.787287,0.210459,baseline
7,0.766167,0.538426,0.652301,0.656396,0.001311,barycentre
8,0.757408,0.658621,0.786661,0.7662,0.012319,partial repair2
9,0.843715,0.674337,0.815176,0.784537,0.164602,reweighting


In [9]:
report.to_csv(path+'/data/report_postprocess_bary'+str(pa)+'.csv',index=None)